In [ ]:
import polar as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Se hace hace una grafica de correlación con las categorias de Minpoint H, usando cada una de las carpetas que hay ECOINVENT 

In [ ]:
# Ruta a tu Excel
ruta_excel = "H Transportation and storage.xlsx"
excel_hojas = ["H Transportation"]

In [ ]:
hojas = [excel_hojas[0]]

In [ ]:
df = pl.read_excel(ruta_excel, sheet_name=excel_hojas[0])

columnas_no_numericas = ["Name", "id", "ubi"]
columnas_numericas = [c for c in df.columns if c not in columnas_no_numericas]

df_pd = df.to_pandas()

In [ ]:
# Filtrar outliers (IQR) solo en columnas numéricas ---
Q1 = df_pd[columnas_numericas].quantile(0.25)
Q3 = df_pd[columnas_numericas].quantile(0.75)
IQR = Q3 - Q1

mask = ((df_pd[columnas_numericas] >= (Q1 - 1.5 * IQR)) & (df_pd[columnas_numericas] <= (Q3 + 1.5 * IQR))).all(axis=1)

df_filtrado = df_pd[mask]
print("Filas originales:", df_pd.shape[0])
print("Filas después de eliminar outliers:", df_filtrado.shape[0])

In [ ]:
# Calcular correlaciones ---
corr_original = df_pd[columnas_numericas].corr(method='pearson')
corr_filtrado = df_filtrado[columnas_numericas].corr(method='pearson')

In [ ]:
#Graficar heatmap de datos originales ---
plt.figure(figsize=(12, 10))
sns.heatmap(corr_original, annot=True, cmap='rainbow', center=0, square=True, fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación - Datos Originales", fontsize=14)
plt.savefig(f"{hojas}_1_scatter.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
#Graficar heatmap de datos filtrados ---
plt.figure(figsize=(12, 10))
sns.heatmap(corr_filtrado, annot=True, cmap='rainbow', center=0, square=True, fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación - Datos Filtrados (sin outliers)", fontsize=14)
plt.savefig(f"{hojas}_2_scatter.png", dpi=300, bbox_inches="tight")
plt.show()